# Scalable Recommendation Retrieval with Locality Sensitive Hashing

From the previous example, we can see that the cost of exhaustive search is linear to the number of items, i.e., $n$ and number of features, i.e., $d$. 

In this part, we will practice to use Locality Sensitive Hashing to speed up the recommendation retrieval task. 

In [ ]:
%matplotlib inline
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from utils.lsh import *
from utils.load_data import *
from utils.pmf import *
from utils.evaluation import *

In [ ]:
#load train/test data and trained pmf model
path_to_model = 'pmf_mvl1m.model'
path_to_train_data = 'train_data'
path_to_test_data  = 'test_data'

pmf_mvl = pickle.load(open(path_to_model, 'rb'))
train   = pickle.load(open(path_to_train_data, 'rb'))
test    = pickle.load(open(path_to_test_data, 'rb'))

## Performance of Linear Scanning Solution

We first measure the precision@10 and recall@10

In [ ]:
# measuring performance of the first 1000 users
topK = 10
data = pmf_mvl.w_Item
queries = pmf_mvl.w_User[:1000,:]

linear_prec, linear_recall = evaluate_topK(test, data, queries, topK)
print('linear_prec@{0} \t linear_recall@{0}'.format(topK))
print('{0}\t{1}'.format(linear_prec, linear_recall))

## Locality Sensitive Hashing

One of the most popular search protocal using Locality Sensitive Hashing structure is Hashtable look-up (illustrated below).  

<img src="resources/images/lsh_retrieval.png" width="600">

In this experiment, we build LSH index on the output of PMF algorithm. You should expect to see the precision and recall degeneration as compared to those of linear scanning solution. Here, we report three values:


1. relative_prec@10 = $\frac{\text{precision@10 of LSH Indexing}}{\text{precision@10 of linear scanning}}$ 
&nbsp;
&nbsp;
&nbsp;  
&nbsp;
2. relative_rec@10    = $\frac{\text{recall@10 of LSH Indexing}}{\text{recall@10 of linear scanning}}$
&nbsp;
&nbsp;
&nbsp;  
&nbsp;
3. touched = $\frac{\text{Average number of investigated items by LSH}}{\text{Total number of items}}$

## Without Transformation
Since Xbox transformation augments user and item vectors to $(d+1)-$dimensional space. For comparison purpose, we append 0s to each user and item vector. 
\begin{equation}
P(y_i) = [y_i, 0] 
\end{equation}

\begin{equation}
Q(x_u) = [x_u, 0]
\end{equation}

With this transformation, we have:

\begin{equation}
Q(x_u)^T.P(y_i) = x_u^T.y_i
\end{equation}

In [ ]:
same_queries = np.concatenate((queries, np.zeros((queries.shape[0], 1))), axis = 1)
same_data = np.concatenate((data, np.zeros((data.shape[0], 1))), axis = 1)

### With Xbox Transformation

Now, before building LSH index, we first apply the Xbox transformation for both user and item vectors. This original maximum inner product search on the original representation becomes the maximum cosine similarity search on the new representation.

\begin{equation}
P(y_i) = [y_i, \sqrt{M^2 - ||y_i||^2}] (M = \max\{||y_i||\})
\end{equation}

\begin{equation}
Q(x_u) = [x_u, 0]
\end{equation}

We have the following observation:

\begin{equation}
\frac{Q(x_u)^T.P(y_i)}{||Q(x_u)||.||P(y_i)||} = \frac{x_u^T.y_i}{M.||x_u||}
\end{equation}

i.e., 

\begin{equation}
\arg\max_{1\leq i\leq n}{x_u^Ty_i} = \arg\max_{1\leq i\leq n}\frac{Q(x_u)^T.P(y_i)}{||Q(x_u)||.||P(y_i)||}
\end{equation}


We Xbox transformation, we effectively convert Maximum Inner Product Search (MIPS) problem to Maximum Cosine Similarity Search (MCCS).

In [ ]:
#apply Xbox transformation
M = np.linalg.norm(data, axis=1) # compute item vector norms
max_norm = max(M) # max item norm

xbox_data = np.concatenate((data, np.sqrt(max_norm**2 - pow(M, 2)).reshape(data.shape[0], -1)), axis = 1)
xbox_queries = np.concatenate((queries, np.zeros((queries.shape[0], 1))), axis = 1)

## Comparing LSH performances with vs. without Xbox transformation

In [ ]:
topK = 10 # top-K value
b_vals = [4, 6, 8] # number of hash function
L_vals = [5, 10] #number of hashtables

print('#table\t #bit\t ?Xbox \t relative_prec@{0} \t relative_recall@{0} \t touched'.format(topK))
for nt in L_vals:
    for b in b_vals: 
        #init lsh index:
        #------ hash-family: the LSH scheme/family 
        #------ k          : number of hash functions
        #------ L          : number of hash tables
        lsh_index = LSHIndex(hash_family = CosineHashFamily(same_data.shape[1]), k = b, L=nt)
        
        #performance without employing Xbox transformation
        print('---------------------------------------------------------------------------------')
        prec_1, recall_1, touched_1 = evaluate_LSHTopK(test, same_data, -same_queries, lsh_index, dot, topK)
        print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}".format(nt, b, 'No',prec_1/linear_prec, recall_1/linear_recall, touched_1)) 
        
        #performance with Xbox transformation
        prec_2, recall_2, touched_2 = evaluate_LSHTopK(test, xbox_data, -xbox_queries, lsh_index, dot, topK)
        print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}".format(nt, b, 'Yes', prec_2/linear_prec, recall_2/linear_recall, touched_2)) 

#table	 #bit	 ?Xbox 	 relative_prec@10 	 relative_recall@10 	 touched
---------------------------------------------------------------------------------
